In [606]:
import pandas as pd
import sqlalchemy as sqla
import pyodbc

from importlib import reload

from country_correction import adjust_country_names
reload(country_correction)

<module 'country_correction' from 'd:\\Github\\pandemic-etl\\country_correction.py'>

In [607]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

DB_USERNAME = os.getenv('DB_USERNAME')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')

FOLDER = 'Dataset'
FILE_COVID = FOLDER + '/COVID_2019.csv'
FILE_EBOLA = FOLDER + '/EBOLA_2014_2016.csv'
FILE_H1N1 = FOLDER + '/H1N1_2009.csv'
FILE_SARS = FOLDER + '/SARS_2003.csv'

In [608]:
connection_url = f"mssql+pymssql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"

engine = sqla.create_engine(connection_url)
try:
    with engine.connect() as conn:
        # result = conn.execute(sqla.text("SELECT * FROM Dim.Date"))
        
        # # Fetch all results
        # for row in result:
        #     print(row)
        pass

except Exception as e:
    print(f"SQLAlchemy connection error: {e}")

# Covid

In [609]:
covid = pd.read_csv(FILE_COVID)
# df.drop([
#         'Date',
#         'Confirmed',
#         'Deaths',
#         'Recovered',
#         'Active'
#     ], axis='columns', inplace=True)
covid.rename(columns={
    'Province/State': "ProvinceState",
    'Country/Region': "Country",
    'Lat': "Latitude",
    'Long': "Longitude",
    'WHO Region': "WHORegion"
}, inplace=True)
covid['Date'] = pd.to_datetime(covid['Date']).dt.strftime('%Y-%m-%d')
adjust_country_names(covid)
covid.drop_duplicates(inplace=True)
covid

# with engine.begin() as conn:
#     df.to_sql(
#         name='Country',
#         schema='Dim',
#         con=conn,
#         index_label='CountryID',
#         if_exists='replace', 
#     )
dim_country = set(covid['Country'].unique())
dim_country
covid

,ProvinceState,Country,Latitude,Longitude,Date,Confirmed,Deaths,Recovered,Active,WHORegion
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,0,Africa
...,...,...,...,...,...,...,...,...,...,...
48802,NaN,Sao Tome and Principe,0.186400,6.613100,2020-07-26,863,14,696,153,Africa
48803,NaN,Yemen,15.552727,48.516388,2020-07-26,1681,479,797,405,Eastern Mediterranean
48804,NaN,Comoros,-11.645500,43.333300,2020-07-26,354,7,328,19,Africa
48805,NaN,Tajikistan,38.861000,71.276100,2020-07-26,7192,59,5970,1163,Europe


# Ebola

In [610]:
ebola = pd.read_csv(FILE_EBOLA)
ebola['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
ebola.rename(columns={
    'Cumulative no. of confirmed, probable and suspected cases':    'Confirmed',
    'Cumulative no. of confirmed, probable and suspected deaths':   'Deaths',
}, inplace=True)
adjust_country_names(ebola)
ebola
new_country = set(ebola['Country'].unique())
diff = new_country - dim_country
diff

set()

# H1N1

In [611]:
h1n1 = pd.read_csv(FILE_H1N1)
h1n1['Date'] = pd.to_datetime(h1n1['Date']).dt.strftime('%Y-%m-%d')
h1n1.rename(columns={
    'Cumulative no. of cases':    'Confirmed',
    'Cumulative no. of deaths':   'Deaths',
}, inplace=True)
adjust_country_names(h1n1)
h1n1
new_country = set(h1n1['Country'].unique())
diff = new_country - dim_country
diff

{'Brunei Darussalam',
 'Cook Island',
 'France, French Polynesia, FOC',
 'France, Martinique, FOC',
 'France, New Caledonia, FOC',
 'French Polynesia, FOC',
 'Guadaloupe, FOC',
 'Guernsey, Crown Dependency',
 'Hong Kong Special Administrative Region',
 'Isle of Man, Crown Dependency',
 'Jersey, Crown Dependency',
 'Martinique, FOC',
 'Netherlands Antilles, Curaçao',
 'Netherlands Antilles, Sint Maarten',
 'Netherlands, Aruba',
 'Netherlands, Curacao, OT',
 'New Caledonia, FOC',
 'Palau',
 'Puerto Rico',
 'Saint Martin, FOC',
 'Samoa',
 'The former Yugoslav Republic of Macedonia',
 'UK',
 'United Kingdom, Isle of Man, Crown Dependency',
 'United Kingdom, Jersey, Crown Dependency',
 'Vanuatu',
 'Virgin Islands',
 'West Bank and Gaza Strip'}

# SARS

In [612]:
sars = pd.read_csv(FILE_SARS)
sars['Date'] = pd.to_datetime(sars['Date']).dt.strftime('%Y-%m-%d')
sars.rename(columns={
    'Cumulative number of case(s)': 'Confirmed',
    'Number of deaths':             'Deaths',
    'Number recovered':             'Recovered',
}, inplace=True)
adjust_country_names(sars)
sars
new_country = set(sars['Country'].unique())
diff = new_country - dim_country
diff

set()

In [613]:
from copy import deepcopy
df = pd.concat([covid,ebola,h1n1,sars], ignore_index=True)
df

,ProvinceState,Country,Latitude,Longitude,Date,Confirmed,Deaths,Recovered,Active,WHORegion
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0.0,0.0,0.0,0.0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0.0,0.0,0.0,0.0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0.0,0.0,0.0,0.0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0.0,0.0,0.0,0.0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0.0,0.0,0.0,0.0,Africa
...,...,...,...,...,...,...,...,...,...,...
56315,NaN,Switzerland,NaN,NaN,2003-07-11,1.0,0.0,1.0,NaN,NaN
56316,NaN,Thailand,NaN,NaN,2003-07-11,9.0,2.0,7.0,NaN,NaN
56317,NaN,United Kingdom,NaN,NaN,2003-07-11,4.0,0.0,4.0,NaN,NaN
56318,NaN,United States,NaN,NaN,2003-07-11,75.0,0.0,67.0,NaN,NaN
